In [2]:
import torch
%load_ext autoreload
%autoreload 2

In [3]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [4]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class = 'sofa')
overfit_dataset = ShapeNet('overfit')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 64

Length of train set: 3173
Length of overfit set: 64


In [5]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[0]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

In [ ]:
##################
#                #
#    TRAINING    #
#                #
##################

In [5]:
# OVERFIT
from training import train
config = {
    'experiment_name': 'overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None

}
train.main(config)

Using device: cuda:0
Loading saved model, latent codes, and latent variances...
Training params: 3
[001/00004] train_loss: 0.010115 kl_loss: 0.007572 normal_loss: 0.002543
[001/00004] train_loss: 0.010374 kl_loss: 0.007749 normal_loss: 0.002626
[002/00004] train_loss: 0.008716 kl_loss: 0.007542 normal_loss: 0.001174
[002/00004] train_loss: 0.008393 kl_loss: 0.007422 normal_loss: 0.000971
[003/00004] train_loss: 0.008175 kl_loss: 0.007302 normal_loss: 0.000872
[003/00004] train_loss: 0.008076 kl_loss: 0.007325 normal_loss: 0.000750
[004/00004] train_loss: 0.007750 kl_loss: 0.007125 normal_loss: 0.000625
[004/00004] train_loss: 0.007689 kl_loss: 0.007174 normal_loss: 0.000515


In [32]:
# AIRPLANE VAD
from training import train
config = {
    'experiment_name': 'airplane_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 4045
Training params: 3
[001/00035] train_loss: 0.114128 kl_loss: 0.161506 normal_loss: 0.109283
[003/00007] train_loss: 0.052188 kl_loss: 0.132147 normal_loss: 0.048223
[004/00043] train_loss: 0.044743 kl_loss: 0.095125 normal_loss: 0.041889
[006/00015] train_loss: 0.039508 kl_loss: 0.071801 normal_loss: 0.037354
[007/00051] train_loss: 0.034843 kl_loss: 0.058168 normal_loss: 0.033097
[009/00023] train_loss: 0.033226 kl_loss: 0.053595 normal_loss: 0.031618
[010/00059] train_loss: 0.030942 kl_loss: 0.050560 normal_loss: 0.029425
[012/00031] train_loss: 0.031112 kl_loss: 0.045498 normal_loss: 0.029747
[014/00003] train_loss: 0.030366 kl_loss: 0.041296 normal_loss: 0.029128
[015/00039] train_loss: 0.030400 kl_loss: 0.038173 normal_loss: 0.029255
[017/00011] train_loss: 0.029179 kl_loss: 0.037097 normal_loss: 0.028066
[018/00047] train_loss: 0.028428 kl_loss: 0.037124 normal_loss: 0.027314
[020/00019] train_loss: 0.027122 kl_loss: 0.038336 normal_loss: 0.0

KeyboardInterrupt: 

In [6]:
# AIRPLANE AD
from training import train
config = {
    'experiment_name': 'airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 4045
Training params: 2
[001/00035] train_loss: 0.100487 kl_loss: 0.000000 normal_loss: 0.100487
[003/00007] train_loss: 0.045317 kl_loss: 0.000000 normal_loss: 0.045317
[004/00043] train_loss: 0.037515 kl_loss: 0.000000 normal_loss: 0.037515
[006/00015] train_loss: 0.032982 kl_loss: 0.000000 normal_loss: 0.032982
[007/00051] train_loss: 0.031851 kl_loss: 0.000000 normal_loss: 0.031851
[009/00023] train_loss: 0.031178 kl_loss: 0.000000 normal_loss: 0.031178
[010/00059] train_loss: 0.029498 kl_loss: 0.000000 normal_loss: 0.029498
[012/00031] train_loss: 0.028000 kl_loss: 0.000000 normal_loss: 0.028000
[014/00003] train_loss: 0.026500 kl_loss: 0.000000 normal_loss: 0.026500
[015/00039] train_loss: 0.025869 kl_loss: 0.000000 normal_loss: 0.025869
[017/00011] train_loss: 0.024928 kl_loss: 0.000000 normal_loss: 0.024928
[018/00047] train_loss: 0.024444 kl_loss: 0.000000 normal_loss: 0.024444
[020/00019] train_loss: 0.023165 kl_loss: 0.000000 normal_loss: 0.0

KeyboardInterrupt: 

In [89]:
# CHAIR
from training import train
config = {
    'experiment_name': 'chair_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 6000
Training params: 2
[001/00005] train_loss: 0.170789 kl_loss: 0.000000 normal_loss: 0.170789
[002/00011] train_loss: 0.119310 kl_loss: 0.000000 normal_loss: 0.119310
[003/00017] train_loss: 0.112543 kl_loss: 0.000000 normal_loss: 0.112543
[004/00023] train_loss: 0.106655 kl_loss: 0.000000 normal_loss: 0.106655
[005/00029] train_loss: 0.104214 kl_loss: 0.000000 normal_loss: 0.104214
[006/00035] train_loss: 0.098748 kl_loss: 0.000000 normal_loss: 0.098748
[007/00041] train_loss: 0.095269 kl_loss: 0.000000 normal_loss: 0.095269
[008/00047] train_loss: 0.090359 kl_loss: 0.000000 normal_loss: 0.090359
[009/00053] train_loss: 0.089151 kl_loss: 0.000000 normal_loss: 0.089151
[010/00059] train_loss: 0.084609 kl_loss: 0.000000 normal_loss: 0.084609
[011/00065] train_loss: 0.083053 kl_loss: 0.000000 normal_loss: 0.083053
[012/00071] train_loss: 0.081139 kl_loss: 0.000000 normal_loss: 0.081139
[013/00077] train_loss: 0.077102 kl_loss: 0.000000 normal_loss: 0.0

KeyboardInterrupt: 

In [315]:
# SOFA & CHAIR VAD
from training import train
config = {
    'experiment_name': 'sofa_chair_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa_chair',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 9173
Training params: 3
[000/00099] train_loss: 0.178056 kl_loss: 0.500034 normal_loss: 0.163055
[001/00055] train_loss: 0.132347 kl_loss: 0.470443 normal_loss: 0.118233
[002/00011] train_loss: 0.124186 kl_loss: 0.438878 normal_loss: 0.111019
[002/00111] train_loss: 0.121336 kl_loss: 0.385560 normal_loss: 0.109769
[003/00067] train_loss: 0.119753 kl_loss: 0.350740 normal_loss: 0.109230
[004/00023] train_loss: 0.117066 kl_loss: 0.325709 normal_loss: 0.107294
[004/00123] train_loss: 0.113976 kl_loss: 0.298345 normal_loss: 0.105026
[005/00079] train_loss: 0.110803 kl_loss: 0.291768 normal_loss: 0.102050
[006/00035] train_loss: 0.109214 kl_loss: 0.287032 normal_loss: 0.100603
[006/00135] train_loss: 0.108182 kl_loss: 0.279403 normal_loss: 0.099800
[007/00091] train_loss: 0.104936 kl_loss: 0.269548 normal_loss: 0.096849
[008/00047] train_loss: 0.101741 kl_loss: 0.267557 normal_loss: 0.093714
[009/00003] train_loss: 0.100989 kl_loss: 0.270261 normal_loss: 0.0

KeyboardInterrupt: 

In [163]:
# SOFA VAD
from training import train
config = {
    'experiment_name': 'sofa_vad_0_03kl',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 3173
Training params: 3
[001/00049] train_loss: 0.158797 kl_loss: 0.485190 normal_loss: 0.144241
[003/00049] train_loss: 0.103193 kl_loss: 0.413880 normal_loss: 0.090777
[005/00049] train_loss: 0.092658 kl_loss: 0.348836 normal_loss: 0.082193
[007/00049] train_loss: 0.089486 kl_loss: 0.300002 normal_loss: 0.080486
[009/00049] train_loss: 0.084510 kl_loss: 0.266648 normal_loss: 0.076511
[011/00049] train_loss: 0.080834 kl_loss: 0.244796 normal_loss: 0.073490
[013/00049] train_loss: 0.075477 kl_loss: 0.232309 normal_loss: 0.068508
[015/00049] train_loss: 0.072623 kl_loss: 0.224983 normal_loss: 0.065873
[017/00049] train_loss: 0.070084 kl_loss: 0.216381 normal_loss: 0.063592
[019/00049] train_loss: 0.068342 kl_loss: 0.208961 normal_loss: 0.062073
[021/00049] train_loss: 0.065641 kl_loss: 0.202974 normal_loss: 0.059552
[023/00049] train_loss: 0.062942 kl_loss: 0.197744 normal_loss: 0.057010
[025/00049] train_loss: 0.061899 kl_loss: 0.192079 normal_loss: 0.0

In [48]:
# SOFA AD
from training import train
config = {
    'experiment_name': 'sofa_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 3173
Training params: 2
[001/00049] train_loss: 0.147046 kl_loss: 0.000000 normal_loss: 0.147046
[003/00049] train_loss: 0.086997 kl_loss: 0.000000 normal_loss: 0.086997
[005/00049] train_loss: 0.078271 kl_loss: 0.000000 normal_loss: 0.078271
[007/00049] train_loss: 0.072125 kl_loss: 0.000000 normal_loss: 0.072125
[009/00049] train_loss: 0.063552 kl_loss: 0.000000 normal_loss: 0.063552
[011/00049] train_loss: 0.060275 kl_loss: 0.000000 normal_loss: 0.060275
[013/00049] train_loss: 0.056823 kl_loss: 0.000000 normal_loss: 0.056823
[015/00049] train_loss: 0.056097 kl_loss: 0.000000 normal_loss: 0.056097
[017/00049] train_loss: 0.051553 kl_loss: 0.000000 normal_loss: 0.051553
[019/00049] train_loss: 0.050349 kl_loss: 0.000000 normal_loss: 0.050349
[021/00049] train_loss: 0.047069 kl_loss: 0.000000 normal_loss: 0.047069
[023/00049] train_loss: 0.044120 kl_loss: 0.000000 normal_loss: 0.044120
[025/00049] train_loss: 0.043236 kl_loss: 0.000000 normal_loss: 0.0

KeyboardInterrupt: 

In [501]:
# SOFA VAD
from training import train
config = {
    'experiment_name': 'car_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 5949
Training params: 3
[001/00006] train_loss: 0.128625 kl_loss: 0.495828 normal_loss: 0.113750
[002/00013] train_loss: 0.064590 kl_loss: 0.449093 normal_loss: 0.051117
[003/00020] train_loss: 0.054780 kl_loss: 0.394209 normal_loss: 0.042954
[004/00027] train_loss: 0.052251 kl_loss: 0.344885 normal_loss: 0.041905
[005/00034] train_loss: 0.047728 kl_loss: 0.301620 normal_loss: 0.038679
[006/00041] train_loss: 0.046487 kl_loss: 0.263708 normal_loss: 0.038576
[007/00048] train_loss: 0.047152 kl_loss: 0.231018 normal_loss: 0.040222
[008/00055] train_loss: 0.045063 kl_loss: 0.204679 normal_loss: 0.038922
[009/00062] train_loss: 0.043648 kl_loss: 0.182405 normal_loss: 0.038176
[010/00069] train_loss: 0.042102 kl_loss: 0.166742 normal_loss: 0.037100
[011/00076] train_loss: 0.040511 kl_loss: 0.154277 normal_loss: 0.035882
[012/00083] train_loss: 0.039628 kl_loss: 0.144000 normal_loss: 0.035308
[013/00090] train_loss: 0.038446 kl_loss: 0.132178 normal_loss: 0.0

In [502]:
# CAR AD
from training import train
config = {
    'experiment_name': 'car_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 5949
Training params: 2
[001/00006] train_loss: 0.112116 kl_loss: 0.000000 normal_loss: 0.112116
[002/00013] train_loss: 0.049877 kl_loss: 0.000000 normal_loss: 0.049877
[003/00020] train_loss: 0.042903 kl_loss: 0.000000 normal_loss: 0.042903
[004/00027] train_loss: 0.040008 kl_loss: 0.000000 normal_loss: 0.040008
[005/00034] train_loss: 0.040174 kl_loss: 0.000000 normal_loss: 0.040174
[006/00041] train_loss: 0.038132 kl_loss: 0.000000 normal_loss: 0.038132
[007/00048] train_loss: 0.034730 kl_loss: 0.000000 normal_loss: 0.034730
[008/00055] train_loss: 0.034613 kl_loss: 0.000000 normal_loss: 0.034613
[009/00062] train_loss: 0.032794 kl_loss: 0.000000 normal_loss: 0.032794
[010/00069] train_loss: 0.031808 kl_loss: 0.000000 normal_loss: 0.031808
[011/00076] train_loss: 0.030735 kl_loss: 0.000000 normal_loss: 0.030735
[012/00083] train_loss: 0.030599 kl_loss: 0.000000 normal_loss: 0.030599
[013/00090] train_loss: 0.029965 kl_loss: 0.000000 normal_loss: 0.0

KeyboardInterrupt: 

In [8]:
# CAR AIRPLANE AD
from training import train
config = {
    'experiment_name': 'car_airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': 'car_airplane_ad',
    'filter_class': 'car_airplane',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 9994
Loading saved model, latent codes, and latent variances...
Training params: 2
[000/00099] train_loss: 0.012841 kl_loss: 0.000000 normal_loss: 0.012841
[001/00042] train_loss: 0.009136 kl_loss: 0.000000 normal_loss: 0.009136
[001/00142] train_loss: 0.009702 kl_loss: 0.000000 normal_loss: 0.009702
[002/00085] train_loss: 0.009820 kl_loss: 0.000000 normal_loss: 0.009820
[003/00028] train_loss: 0.010000 kl_loss: 0.000000 normal_loss: 0.010000
[003/00128] train_loss: 0.009638 kl_loss: 0.000000 normal_loss: 0.009638
[004/00071] train_loss: 0.009730 kl_loss: 0.000000 normal_loss: 0.009730
[005/00014] train_loss: 0.009661 kl_loss: 0.000000 normal_loss: 0.009661
[005/00114] train_loss: 0.009315 kl_loss: 0.000000 normal_loss: 0.009315
[006/00057] train_loss: 0.009637 kl_loss: 0.000000 normal_loss: 0.009637
[007/00000] train_loss: 0.009447 kl_loss: 0.000000 normal_loss: 0.009447
[007/00100] train_loss: 0.009116 kl_loss: 0.000000 normal_loss: 0.009116
[008/000

In [10]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [7]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

def visualize_dataset_sample(filter_class, index):
    dataset = ShapeNet('train', filter_class = filter_class)
    sample = dataset[index]
    input_mesh = marching_cubes(sample['target_df'], level=1)
    visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

def visualize_ad(experiment, index):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x = latent_vectors[index].unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_vad(experiment, index):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    log_vars = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x = latent_vectors[index]
    Dist = dist.Normal(x, torch.exp(log_vars[index]))
    x_vad = Dist.rsample().unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x_vad)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_vad_norm(experiment):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Sample
    # Dist = dist.Normal(torch.zeros(256), torch.ones(256))
    x_vad = torch.randn(256).unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x_vad)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_interpolation_ad(experiment, index1, index2, a1=0.5, a2=0.5):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x1 = latent_vectors[index1]
    x2 = latent_vectors[index2]
    x = (a1*x1 + a2*x2).unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

In [43]:
index = 342
index1 = 312
index2 = 33
a1 = 0.5
a2 = 1 - a1
# experiment = "sofa_ad"
# filter_class = "sofa"
experiment = "airplane_vad"
# experiment2 = "sofa_vad_0_03kl"
filter_class = "car_airplane"
#-------
# visualize_vad_norm(experiment2)
visualize_vad_norm(experiment)
# visualize_ad(experiment, index)
#-------
# visualize_vad_norm(experiment)
# visualize_vad_norm(experiment2)
# visualize_ad(experiment2, index)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad(experiment, index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

In [11]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

# load model
name_temp = 'sofa_ad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')
Dist = dist.Normal(latent_vectors[1002], torch.exp(log_vars[1002]))
Dist2 = dist.Normal(latent_vectors[1000], torch.exp(log_vars[1000]))
#x_vad_n = torch.randn(256, device='cpu')
x_vad = Dist.rsample()#.unsqueeze(0)
x_vad2 = Dist2.rsample()#.unsqueeze(0)

a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((x_vad*a1 + x_vad2*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

output_meshes = model(x_vad.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(x_vad2.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)


Output()

Output()

Output()

In [62]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[5]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
# load model
name_temp = 'generalization_airplane_no_vad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))

# load latent codes and latent variances
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')

x_vad_n = x_vad_n.unsqueeze(0)
output_meshes = model(latent_vectors[5].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(latent_vectors[105].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)
a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((latent_vectors[f1]*a1 + latent_vectors[f2]*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

Output()

Output()

Output()